# Grouping and Summarising Dataframes

Grouping and aggregation are some of the most frequently used operations in data analysis, especially while doing exploratory data analysis (EDA), where comparing summary statistics across groups of data is common.

For e.g., in the retail sales data we are working with, you may want to compare the average sales of various regions, or compare the total profit of two customer segments.

Grouping analysis can be thought of as having three parts:
1. **Splitting** the data into groups (e.g. groups of customer segments, product categories, etc.)
2. **Applying** a function to each group (e.g. mean or total sales of each customer segment)
3. **Combining** the results into a data structure showing the summary statistics

Let's work through some examples.

In [1]:
# Loading libraries and files
import numpy as np
import pandas as pd

market_df = pd.read_csv("../market_fact.csv")
customer_df = pd.read_csv("../cust_dimen.csv")
product_df = pd.read_csv("../prod_dimen.csv")
shipping_df = pd.read_csv("../shipping_dimen.csv")
orders_df = pd.read_csv("../orders_dimen.csv")

Say you want to understand how well or poorly the business is doing in various customer segments, regions, product categories etc. Specifically, you want to identify areas of business where you are incurrring heavy losses, and want to take action accordingly.

To do that, we will answer questions such as:
* Which customer segments are the least profitable?
* Which product categories and sub-categories are the least profitable?
* Customers in which geographic region cause the most losses?
* Etc.

First, we will merge all the dataframes, so we have all the data in one ```master_df```.

In [3]:
# Merging the dataframes one by one
df_1 = pd.merge(market_df, customer_df, how='inner', on='Cust_id')
df_2 = pd.merge(df_1, product_df, how='inner', on='Prod_id')
df_3 = pd.merge(df_2, shipping_df, how='inner', on='Ship_id')
master_df = pd.merge(df_3, orders_df, how='inner', on='Ord_id')

master_df.head()

,Ord_id,Prod_id,Ship_id,Cust_id,Sales,Discount,Order_Quantity,Profit,Shipping_Cost,Product_Base_Margin,...,Region,Customer_Segment,Product_Category,Product_Sub_Category,Order_ID_x,Ship_Mode,Ship_Date,Order_ID_y,Order_Date,Order_Priority
0,Ord_5446,Prod_16,SHP_7609,Cust_1818,136.81,0.01,23,-30.51,3.60,0.56,...,WEST,CORPORATE,OFFICE SUPPLIES,"SCISSORS, RULERS AND TRIMMERS",36262,REGULAR AIR,28-07-2010,36262,27-07-2010,NOT SPECIFIED
1,Ord_5446,Prod_4,SHP_7610,Cust_1818,4701.69,0.00,26,1148.90,2.50,0.59,...,WEST,CORPORATE,TECHNOLOGY,TELEPHONES AND COMMUNICATION,36262,EXPRESS AIR,27-07-2010,36262,27-07-2010,NOT SPECIFIED
2,Ord_5446,Prod_6,SHP_7608,Cust_1818,164.02,0.03,23,-47.64,6.15,0.37,...,WEST,CORPORATE,OFFICE SUPPLIES,PAPER,36262,EXPRESS AIR,28-07-2010,36262,27-07-2010,NOT SPECIFIED
3,Ord_2978,Prod_16,SHP_4112,Cust_1088,305.05,0.04,27,23.12,3.37,0.57,...,ONTARIO,HOME OFFICE,OFFICE SUPPLIES,"SCISSORS, RULERS AND TRIMMERS",37863,REGULAR AIR,26-02-2011,37863,24-02-2011,HIGH
4,Ord_5484,Prod_16,SHP_7663,Cust_1820,322.82,0.05,35,-17.58,3.98,0.56,...,WEST,CONSUMER,OFFICE SUPPLIES,"SCISSORS, RULERS AND TRIMMERS",53026,REGULAR AIR,03-03-2012,53026,26-02-2012,LOW


#### Step 1. Grouping using ```df.groupby()```

Typically, you group the data using a categorical variable, such as customer segments, product categories, etc. This creates as many subsets of the data as there are levels in the categorical variable.

For example, in this case, we will group the data along ```Customer_Segment```.

In [ ]:
# Which customer segments are the least profitable?

# Step 1. Grouping: First, we will group the dataframe by customer segments
df_by_segment = master_df.groupby('Customer_Segment')
df_by_segment.head()

Note that ```df.groupby``` returns a DataFrameGroupBy object.

#### Step 2. Applying a Function

After grouping, you apply a function to a **numeric variable**, such as ```mean(Sales)```, ```sum(Profit)```, etc.

In [ ]:
# Step 2. Applying a function
# We can choose aggregate functions such as sum, mean, median, etc.
df_by_segment['Profit'].sum()

Customer_Segment
CONSUMER          287959.94
CORPORATE         599746.00
HOME OFFICE       318354.03
SMALL BUSINESS    315708.01
Name: Profit, dtype: float64

Notice that we have indexed the ```Profit``` column in the DataFrameGroupBy object exactly as we index a normal column in a dataframe. Alternatively, you could also use ```df_by_segment.Profit```.

In [ ]:
# Alternatively
df_by_segment.Profit.sum()

Customer_Segment
CONSUMER          287959.94
CORPORATE         599746.00
HOME OFFICE       318354.03
SMALL BUSINESS    315708.01
Name: Profit, dtype: float64

So this tells us that profits are the least in the CONSUMER segment, and highest in the CORPORATE segment.

In [ ]:
# For better readability, you may want to sort the summarised series:
df_by_segment.Profit.sum().sort_values(ascending = False)

Customer_Segment
CORPORATE         599746.00
HOME OFFICE       318354.03
SMALL BUSINESS    315708.01
CONSUMER          287959.94
Name: Profit, dtype: float64

#### Step 3. Combining the results into a Data Structure

You can optionally show the results as a dataframe.

In [7]:
# Converting to a df
pd.DataFrame(df_by_segment['Profit'].sum())

,Profit
Customer_Segment,
CONSUMER,287959.94
CORPORATE,599746.00
HOME OFFICE,318354.03
SMALL BUSINESS,315708.01


In [8]:
# Let's go through some more examples
# E.g.: Which product categories are the least profitable?

# 1. Group by product category
by_product_cat = master_df.groupby('Product_Category')

In [9]:
# 2. This time, let's compare average profits
# Apply mean() on Profit
by_product_cat['Profit'].mean()

Product_Category
FURNITURE           68.116607
OFFICE SUPPLIES    112.369074
TECHNOLOGY         429.207516
Name: Profit, dtype: float64

FURNITURE is the least profitable, TECHNOLOGY the most. Let's see which product sub-cetgories within FURNITURE are less profitable.

In [10]:
# E.g.: Which product categories and sub-categories are the least profitable?
# 1. Group by category and sub-category
by_product_cat_subcat = master_df.groupby(['Product_Category', 'Product_Sub_Category'])
by_product_cat_subcat['Profit'].mean()

Product_Category  Product_Sub_Category          
FURNITURE         BOOKCASES                         -177.683228
                  CHAIRS & CHAIRMATS                 387.693601
                  OFFICE FURNISHINGS                 127.446612
                  TABLES                            -274.411357
OFFICE SUPPLIES   APPLIANCES                         223.866498
                  BINDERS AND BINDER ACCESSORIES     335.970918
                  ENVELOPES                          195.864228
                  LABELS                              47.490174
                  PAPER                               36.949551
                  PENS & ART SUPPLIES                 11.950679
                  RUBBER BANDS                        -0.573575
                  SCISSORS, RULERS AND TRIMMERS      -54.161458
                  STORAGE & ORGANIZATION              12.205403
TECHNOLOGY        COMPUTER PERIPHERALS               124.389815
                  COPIERS AND FAX                   192

Thus, within FURNITURE, TABLES are the least profitable, followed by BOOKCASES.

In [13]:
# Recall the df.describe() method?
# To apply multiple functions simultaneously, you can use the describe() function on the grouped df object
by_product_cat_subcat['Profit'].describe()

count         mean  \
Product_Category Product_Sub_Category                                  
FURNITURE        BOOKCASES                        189.0  -177.683228   
                 CHAIRS & CHAIRMATS               386.0   387.693601   
                 OFFICE FURNISHINGS               788.0   127.446612   
                 TABLES                           361.0  -274.411357   
OFFICE SUPPLIES  APPLIANCES                       434.0   223.866498   
                 BINDERS AND BINDER ACCESSORIES   915.0   335.970918   
                 ENVELOPES                        246.0   195.864228   
                 LABELS                           288.0    47.490174   
                 PAPER                           1225.0    36.949551   
                 PENS & ART SUPPLIES              633.0    11.950679   
                 RUBBER BANDS                     179.0    -0.573575   
                 SCISSORS, RULERS AND TRIMMERS    144.0   -54.161458   
                 STORAGE & ORGANIZATION           546.0    12.205403   
TECHNOLOGY       COMPUTER PERIPHERALS             758.0   124.389815   
                 COPIERS AND FAX                   87.0  1923.695287   
                 OFFICE MACHINES                  337.0   913.094748   
                 TELEPHONES AND COMMUNICATION     883.0   358.948607   

                                                         std       min  \
Product_Category Product_Sub_Category                                    
FURNITURE        BOOKCASES                       1707.455501 -11053.60   
                 CHAIRS & CHAIRMATS              1482.276988  -3404.24   
                 OFFICE FURNISHINGS               463.997735  -1570.32   
                 TABLES                          1148.310769  -6474.65   
OFFICE SUPPLIES  APPLIANCES                       817.377547  -2172.14   
                 BINDERS AND BINDER ACCESSORIES  1349.974000   -961.50   
                 ENVELOPES                        479.703533   -201.60   
                 LABELS                           136.013924   -223.50   
                 PAPER                            217.200169   -331.63   
                 PENS & ART SUPPLIES               77.341605   -216.66   
                 RUBBER BANDS                      27.420725   -107.00   
                 SCISSORS, RULERS AND TRIMMERS    202.203379  -1759.58   
                 STORAGE & ORGANIZATION           848.234047  -2175.09   
TECHNOLOGY       COMPUTER PERIPHERALS             465.143093   -807.59   
                 COPIERS AND FAX                 4660.287752 -11861.46   
                 OFFICE MACHINES                 3627.391357 -14140.70   
                 TELEPHONES AND COMMUNICATION     642.970292   -888.16   

                                                      25%      50%        75%  \
Product_Category Product_Sub_Category                                           
FURNITURE        BOOKCASES                      -662.8000 -305.980   -78.3600   
                 CHAIRS & CHAIRMATS             -300.5225  -64.580   761.3175   
                 OFFICE FURNISHINGS              -34.0750   24.955   171.2975   
                 TABLES                         -694.3300 -352.960    31.2100   
OFFICE SUPPLIES  APPLIANCES                      -82.1000    6.375   448.6350   
                 BINDERS AND BINDER ACCESSORIES  -64.6400   -9.720    72.6650   
                 ENVELOPES                        -6.6925   39.315   204.1200   
                 LABELS                           10.5625   35.770    58.7400   
                 PAPER                           -69.2900  -14.350    55.8200   
                 PENS & ART SUPPLIES             -15.0300    0.820    24.6000   
                 RUBBER BANDS                    -11.2900   -1.720    11.7750   
                 SCISSORS, RULERS AND TRIMMERS   -64.6675  -31.015    -2.5575   
                 STORAGE & ORGANIZATION         -232.1675  -52.410    33.8725   
TECHNOLOGY       COMPUTER PERIPHERALS            -79.1475  

In [12]:
# Some other summary functions to apply on groups
by_product_cat_subcat['Profit'].count()

Product_Category  Product_Sub_Category          
FURNITURE         BOOKCASES                          189
                  CHAIRS & CHAIRMATS                 386
                  OFFICE FURNISHINGS                 788
                  TABLES                             361
OFFICE SUPPLIES   APPLIANCES                         434
                  BINDERS AND BINDER ACCESSORIES     915
                  ENVELOPES                          246
                  LABELS                             288
                  PAPER                             1225
                  PENS & ART SUPPLIES                633
                  RUBBER BANDS                       179
                  SCISSORS, RULERS AND TRIMMERS      144
                  STORAGE & ORGANIZATION             546
TECHNOLOGY        COMPUTER PERIPHERALS               758
                  COPIERS AND FAX                     87
                  OFFICE MACHINES                    337
                  TELEPHONES AND COMMUN

In [ ]:
by_product_cat['Profit'].min()

Product_Category
FURNITURE         -11053.60
OFFICE SUPPLIES    -2175.09
TECHNOLOGY        -14140.70
Name: Profit, dtype: float64

In [ ]:
# E.g. Customers in which geographic region are the least profitable?
master_df.groupby('Region').Profit.mean()

Region
ATLANTIC                 221.259870
NORTHWEST TERRITORIES    255.464670
NUNAVUT                   35.963418
ONTARIO                  189.960865
PRARIE                   188.253294
QUEBEC                   179.803649
WEST                     149.175595
YUKON                    136.253155
Name: Profit, dtype: float64

In [ ]:
# Note that the resulting object is a Series, thus you can perform vectorised computations on them

# E.g. Calculate the Sales across each region as a percentage of total Sales
# You can divide the entire series by a number (total sales) easily
(master_df.groupby('Region').Sales.sum() / sum(master_df['Sales']))*100

Region
ATLANTIC                 13.504305
NORTHWEST TERRITORIES     5.369193
NUNAVUT                   0.780233
ONTARIO                  20.536970
PRARIE                   19.022396
QUEBEC                   10.124936
WEST                     24.119372
YUKON                     6.542595
Name: Sales, dtype: float64

The regions ONTARIO, WEST and PRARIE comprise of about 64% of the sales.

Until now, we've been working with the data without making changes or additions to it. In the next section, we will create new columns, alter existing columns and apply some more grouping and summarising.



## **Module Exercise :**

In [14]:
import pandas as pd
df = pd.read_csv('https://query.data.world/s/vBDCsoHCytUSLKkLvq851k2b8JOCkF')
#Type your groupby command here
df_1 = df.groupby(['month','day'])#Type your code to find the mean of columns 'rain' and 'wind'

df_2 = df_1[['rain','wind']].mean()

print(df_2.head(20))

               rain      wind
month day                    
apr   fri  0.000000  3.100000
      mon  0.000000  3.100000
      sat  0.000000  4.500000
      sun  0.000000  5.666667
      thu  0.000000  5.800000
      wed  0.000000  2.700000
aug   fri  0.066667  4.766667
      mon  0.000000  2.873333
      sat  0.000000  4.310345
      sun  0.025000  4.417500
      thu  0.000000  3.503846
      tue  0.300000  4.567857
      wed  0.000000  3.520000
dec   fri  0.000000  4.900000
      mon  0.000000  8.500000
      sun  0.000000  8.500000
      thu  0.000000  4.900000
      tue  0.000000  8.500000
      wed  0.000000  8.000000
feb   fri  0.000000  4.820000
